# Pathfinder tests

The goal of this module is to test the functionality of the pathfinder modules using the pre-saved data from the fullscan pathfinder.

In [5]:
from mpl_toolkits import mplot3d
import sys
sys.path.append('../')
print(sys.path)
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import os
import json
from DemoPathfinder import *

In [4]:
path = r"C:\Users\ander\Documents\COBOT-Transducer-Control-Code\Rebuilding_Everything"
current_file = r"\2022-05-05_16-42-57_test.json"

with open(path + current_file, 'r') as outfile:
    point_mags = json.load(outfile)


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\ander\\Documents\\COBOT-Transducer-Control-Code\\Rebuilding_Everything\\2022-05-05_16-42-57_test.json'

In [ ]:
class item:
    def __init__(self, a):
        fe = 1
        fi = 2
        fo = 3
        fum = a
        t = "asfasdf"
    
    def print_json(self):
        print(json.dumps(self))

b = item(("x", "y", "Rz"))
b.print_json()